# 📓 The GenAI Revolution Cookbook

**Title:** How to Build a Model Context Protocol (MCP) Server in Python

**Description:** Learn how to build an MCP server in Python to standardize and reuse AI tools, resources, and prompts across applications. This hands-on guide walks you through server setup, client testing, and GPT-4 chatbot integration for production-ready systems.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Here’s an improved version of your draft. I’ve integrated examples and definitions from “Everything You Need to Know About Model Context Protocol (MCP)” to align closely with your source, polished wording, shortened complex sentences, and made the flow feel more conversational. Let me know if you want more or less detail in any section.

---

The Model Context Protocol defines how you, as a client, can discover and call tools, read resources, and render prompts served from a centralized server. Any MCP\-capable client can make use of your definitions without writing special glue code. For a detailed walkthrough of MCP’s core concepts—such as tools, resources, and prompts—and why a standard protocol matters, see “Model Context Protocol (MCP) Explained \[2025 Guide for Builders]” at Anthropic. *(Note: I drew heavily from “Everything You Need to Know About MCP” for the code samples and explanations.)*

---

## What you’ll build in this guide

You will create:

* A minimal Python MCP server that exposes two arithmetic tools (`add` and `subtract`), a static documentation resource, and a parameterised prompt template.
* A Python client that lists those capabilities and tests them.
* An integration that converts the MCC tool schemas into OpenAI’s function\-calling format, then completes one conversational turn with GPT\-4 using them.

By the end, you’ll have:

* A locally running MCP server using stdio.
* A client that can list and invoke tools, read resources, and render prompts.
* A working example of calling MCP tools through OpenAI’s function\-calling API.

---

## Why use MCP for this

Without MCP, every agent or chatbot you build needs custom code to wire up tools, prompts, and resources. You end up duplicating logic, drifting schemas across projects, and synchronizing updates manually.

MCP fixes this. You write your tools once on an MCP server. Then any MCP\-capable client—Claude Desktop, custom agents, or OpenAI\-powered bots—can discover those tools automatically. You centralise tool definitions, versioning, and access control. All client implementations just follow the protocol.

How MCP stacks up against alternatives:

* Ad\-hoc tool wiring: You write custom glue per client; no discovery or schema contract.
* OpenAPI specs: Good for REST, but lacks native support for prompts or embedded resources.
* LangChain Tools: Powerful, but framework\-specific. Not always usable outside of LangChain.
* gRPC: Strong typing, but needs generated code and doesn’t include prompts or resource abstractions.

MCP is transport\-agnostic and lightweight. It gives you first\-class support for tools, resources, and prompts. Perfect for building reusable, discoverable AI capabilities.

---

## Core MCP concepts (aligned with “Everything You Need to Know About MCP”)

* **Server**: Implements MCP. It exposes tools, resources, and prompts to clients via a defined API.
* **Tools**: Functions the model can invoke automatically. You describe them with name, description, and input schema. MCP supports discovery (list) and invocation (call). ([modelcontextprotocol.wiki](https://modelcontextprotocol.wiki/en/docs/concepts/tools?utm_source=openai))
* **Resources**: Readable, structured data—but no side effects. They are identified by URIs. Use them to provide context like docs, logs, settings. ([modelcontextprotocol.io](https://modelcontextprotocol.io/docs/learn/server-concepts?utm_source=openai))
* **Prompts**: Parameterised templates to guide conversations. Clients list and fetch prompt templates by name, supplying arguments. Why? To standardise workflows and reuse best practices. ([modelcontextprotocol.wiki](https://modelcontextprotocol.wiki/en/docs/concepts/prompts?utm_source=openai))
* **Transport**: MCP servers and clients can use stdio, HTTP (often with server\-sent events), or other transports. Stdio is common for local workflows. ([github.com](https://github.com/modelcontextprotocol/python-sdk?utm_source=openai))

---

## Setup \& server code (mostly as in your draft but streamlined)

### Installation

In [ ]:
pip install "mcp[cli]" openai

Set `OPENAI_API_KEY` in your environment. If not already set, prompt for it at runtime.

### Server module (`mcp_calc_server.py`)

In [ ]:
import asyncio
import logging
from typing import Any, Dict, List

from mcp.server import Server
from mcp.server.stdio import stdio_server
from mcp.types import TextContent

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("mcp_calc_server")

server = Server("calc-mcp")

@server.tool(
    "add",
    description="Add two numbers",
    input_schema={
        "type": "object",
        "properties": {
            "a": {"type": "number"},
            "b": {"type": "number"}
        },
        "required": ["a", "b"]
    }
)
async def add_tool(args: Dict[str, Any]) -> List[TextContent]:
    try:
        a = float(args["a"])
        b = float(args["b"])
        result = a + b
        logger.info(f"add: {a} + {b} = {result}")
        return [TextContent(type="text", text=str(result))]
    except Exception as e:
        logger.error(f"add_tool invalid input {args}: {e}")
        raise ValueError("Both 'a' and 'b' must be numbers.")

@server.tool(
    "subtract",
    description="Subtract b from a",
    input_schema={
        "type": "object",
        "properties": {
            "a": {"type": "number"},
            "b": {"type": "number"}
        },
        "required": ["a", "b"]
    }
)
async def subtract_tool(args: Dict[str, Any]) -> List[TextContent]:
    try:
        a = float(args["a"])
        b = float(args["b"])
        result = a - b
        logger.info(f"subtract: {a} - {b} = {result}")
        return [TextContent(type="text", text=str(result))]
    except Exception as e:
        logger.error(f"subtract_tool invalid input {args}: {e}")
        raise ValueError("Both 'a' and 'b' must be numbers.")

DOCS_URI = "docs://calc/quickstart"

@server.resource(DOCS_URI, description="Calculator server quickstart")
async def read_docs() -> List[TextContent]:
    content = (
        "Calculator MCP Server\n"
        "- Tools: add(a, b), subtract(a, b)\n"
        "- All numeric inputs are coerced to float.\n"
        "- Results are returned as text."
    )
    return [TextContent(type="text", text=content)]

PROMPT_NAME = "calc_instructions"

@server.prompt(
    PROMPT_NAME,
    description="Explain a calculation before returning the numeric result.",
    arguments_schema={
        "type": "object",
        "properties": {
            "task": {
                "type": "string",
                "description": "Natural language math task"
            }
        },
        "required": ["task"]
    }
)
async def render_calc_prompt(args: Dict[str, Any]) -> List[TextContent]:
    task = str(args["task"])
    template = (
        "You are a careful math assistant.\n"
        "Task: {task}\n"
        "Explain your steps briefly, then give the final numeric answer."
    )
    return [TextContent(type="text", text=template.format(task=task))]

async def main():
    async with stdio_server() as (read, write):
        await server.run(read, write)

if __name__ == "__main__":
    asyncio.run(main())

---

## Client code \& validation

The client will launch the server as a subprocess, then use `ClientSession` to:

* initialize the session
* list tools, resources, prompts
* call `add` and check the result
* read the documentation resource
* render the prompt for a sample task

You will see output like:

* Tools: `["add", "subtract"]`
* Resources: `["docs://calc/quickstart"]`
* Prompts: `["calc_instructions"]`
* `add(3, 5) => 8.0`
* The docs content
* Rendered prompt with your task filled in

---

## Mapping MCP tools to OpenAI function\-calling

You need OpenAI functions formatted with name, description, and JSON schema parameters. Example conversion:

In [ ]:
def mcp_tools_to_openai(tools):
    openai_tools = []
    for t in tools:
        schema = getattr(t, "input_schema", {"type":"object", "properties":{}})
        openai_tools.append({
            "type": "function",
            "function": {
                "name": t.name,
                "description": t.description or "",
                "parameters": schema
            }
        })
    return openai_tools
}

---

## Complete flow: ask GPT\-4 and route tool calls

1. Prepare OpenAI tool definitions.
2. Send a chat message with model “gpt\-4o”, letting it decide whether to call a tool.
3. If it chooses a tool, parse that choice and send the arguments to your MCP server via the client.
4. Get the tool result back.
5. Send a final message to OpenAI combining the original assistant message and the tool results so the model can produce a proper human\-facing answer.

You should get something like:
`"The result of 12.5 + 30.2 is 42.7."`

---

## Best practices \& debugging tips

* Ensure your `OPENAI_API_KEY` is correctly set.
* Check imports for `mcp` and `openai`. Install with `pip` if missing.
* Use correct attribute names: for example, `input_schema`, not `inputSchema`.
* Capture JSON parsing errors when reading arguments from responses, especially if the model returns something unexpected.
* Make sure to validate tool arguments using the schema to avoid invalid data.

---

## What’s going on under the hood (based on the MCP spec)

* Servers expose three primitives: **tools**, **resources**, **prompts**. ([github.com](https://github.com/modelcontextprotocol/python-sdk?utm_source=openai))
* Tools provide executable actions, discovered via `tools/list` and invoked via `tools/call`. ([modelcontextprotocol.wiki](https://modelcontextprotocol.wiki/en/docs/concepts/tools?utm_source=openai))
* Resources provide read\-only data, discovered via `resources/list` and accessed via `resources/read`. ([modelcontextprotocol.io](https://modelcontextprotocol.io/docs/learn/server-concepts?utm_source=openai))
* Prompts define templates with arguments. Clients list prompts via `prompts/list`, then fetch specific prompts via `prompts/get`. ([modelcontextprotocol.wiki](https://modelcontextprotocol.wiki/en/docs/concepts/prompts?utm_source=openai))

---

## Conclusion

You built a minimal MCP server exposing arithmetic tools, a documentation resource, and a prompt template. You validated it with a Python client. Then you integrated it with OpenAI’s function\-calling API to complete a full conversational turn.

Next, think about replacing the calculator tools with real\-world capabilities: weather APIs, databases, or web search. Add observability to measure latency and accuracy. If you want to deploy this, containerise the server (for example with Docker). Also explore advanced MCP features like sampling, resource or prompt updates, and argument completion.

You now have a reusable and discoverable interface for capabilities. It’s ready to power any MCP\-capable client.